In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model = 'gpt-4o')

In [3]:
from langchain_community.document_loaders import JSONLoader
loader = JSONLoader(file_path='Patient_ID_BABY JESUS_2024-09-19T01_46_50.449Z.json', jq_schema='.', text_content=False)
docs  = loader.load()

In [4]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 100, add_start_index = True)
all_splits = text_splitter.split_documents(docs)

In [5]:
from langchain_openai import OpenAIEmbeddings

embed = OpenAIEmbeddings(show_progress_bar=True, request_timeout= 10000)

In [6]:
from langchain_chroma import Chroma

vectorstore = Chroma.from_documents(documents=all_splits, embedding=embed, persist_directory= './DEMON_db')

  0%|          | 0/1 [00:00<?, ?it/s]

In [7]:
retriever = vectorstore.as_retriever(search_type="similarity",search_kwargs={'k': 6})

In [8]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [9]:
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

template = """Use the following pieces of context to summarize the data at the end.
If you dont know, just say you dont know. 
Pretend you are a health care worker at the ned of your shift. 
Now is the time for patient handover. Explain and summarize to the next nurse the status of the patient described in the JSON file.
In the case of irregular vital sign fluctuations, flag it and mention it 

Always say "The End!", in a new line at the end of the answer.

{context}

Question: {question}

Helpful Answer:"""


custom_rag_prompt = PromptTemplate.from_template(template)

rag_chain = ({"context": retriever | format_docs, "question": RunnablePassthrough()} | custom_rag_prompt | llm | StrOutputParser())

In [10]:
response = rag_chain.invoke("Summarize")
print(response)

  0%|          | 0/1 [00:00<?, ?it/s]

During the shift, the patient "BABY JESUS" was continuously monitored. The baby was not physically present ('no_baby' in occupancy). The environment remained consistent with low lighting, supine position, and covered with a blanket. No interventions were needed, and the NIPS pain assessment consistently indicated no pain.

Vital sign observations included:

- **Heart Rate:** There were significant fluctuations in the heart rate. It ranged from a low of 80 bpm to a high of 159 bpm. Notable fluctuations occurred, such as the drop from 142 bpm to 87 bpm and subsequent rise to 154 bpm.
- **Respiratory Rate:** The respiratory rate remained steady at 40 breaths per minute.
- **Body Temperature:** Consistent at 37°C throughout the observations.

Overall, the main concern is the irregular fluctuations in heart rate, which should be flagged for further monitoring and evaluation.

The End!
